# 🇳🇵 Nepali Chatbot using Pre-trained Models
## Building a Conversational AI with HuggingFace Transformers


---

### Pre-trained Models Used:
1. **Sakonii/distilgpt2-nepali** - Nepali GPT-2 for text generation
2. **NepBERTa/NepBERTa** - Nepali BERT for understanding
3. **google/mt5-small** - Multilingual T5 for translation/generation
4. **facebook/mbart-large-50** - For multilingual conversations

### What We'll Build:
- Text generation chatbot using Nepali GPT-2
- Question-answering system
- Retrieval-based chatbot with semantic similarity
- Fine-tuning on custom conversation data

## 1. Setup and Installation

In [ ]:
# Install required packages
#!pip install -q transformers torch sentencepiece accelerate datasets

In [ ]:
# Import libraries
import torch
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModel,
    AutoModelForSeq2SeqLM,
    pipeline,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
import warnings
warnings.filterwarnings('ignore')

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Load Pre-trained Nepali GPT-2 Model

We'll use **Sakonii/distilgpt2-nepali** - a GPT-2 model pre-trained on 13+ million Nepali text sequences.

In [ ]:
# Load Nepali GPT-2 model
print("Loading Nepali GPT-2 model...")

MODEL_NAME = "Sakonii/distilgpt2-nepali"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model = model.to(device)

# Set pad token if not set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Model loaded: {MODEL_NAME}")
print(f"   Vocabulary size: {tokenizer.vocab_size}")
print(f"   Model parameters: {model.num_parameters():,}")

In [ ]:
# Test basic text generation
def generate_text(prompt, max_length=100, temperature=0.8, top_p=0.9, num_return_sequences=1):
    """
    Generate Nepali text using the pre-trained model.

    Args:
        prompt: Input text in Nepali
        max_length: Maximum length of generated text
        temperature: Controls randomness (lower = more focused)
        top_p: Nucleus sampling parameter
        num_return_sequences: Number of sequences to generate

    Returns:
        Generated text(s)
    """
    # Encode input
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            num_return_sequences=num_return_sequences,
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=2
        )

    # Decode
    generated_texts = []
    for output in outputs:
        text = tokenizer.decode(output, skip_special_tokens=True)
        generated_texts.append(text)

    return generated_texts[0] if num_return_sequences == 1 else generated_texts

# Test generation
print("Testing text generation...")
test_prompts = [
    "नेपाल एक सुन्दर",
    "काठमाडौं शहरमा",
    "आज मौसम"
]

for prompt in test_prompts:
    generated = generate_text(prompt, max_length=50)
    print(f"\nPrompt: {prompt}")
    print(f"Generated: {generated}")

## 3. Create Chatbot Class using Pre-trained Model

In [ ]:
class NepaliGPTChatbot:
    """
    Nepali Chatbot using pre-trained GPT-2 model.
    """

    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.conversation_history = []

        # Chat templates
        self.user_prefix = "प्रयोगकर्ता: "
        self.bot_prefix = "बोट: "

    def generate_response(self, user_input, max_length=100, temperature=0.7):
        """
        Generate a response to user input.
        """
        # Create prompt with conversation context
        prompt = f"{self.user_prefix}{user_input}\n{self.bot_prefix}"

        # Add recent history for context
        if self.conversation_history:
            history_text = "\n".join(self.conversation_history[-4:])  # Last 2 exchanges
            prompt = history_text + "\n" + prompt

        # Encode
        inputs = self.tokenizer.encode(prompt, return_tensors='pt').to(self.device)

        # Generate
        with torch.no_grad():
            outputs = self.model.generate(
                inputs,
                max_length=len(inputs[0]) + max_length,
                temperature=temperature,
                top_p=0.9,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                no_repeat_ngram_size=2,
                early_stopping=True
            )

        # Decode and extract response
        full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract just the bot's response
        response = full_response.split(self.bot_prefix)[-1].strip()

        # Clean up - take first sentence/line
        if "\n" in response:
            response = response.split("\n")[0]
        if self.user_prefix in response:
            response = response.split(self.user_prefix)[0]

        # Update history
        self.conversation_history.append(f"{self.user_prefix}{user_input}")
        self.conversation_history.append(f"{self.bot_prefix}{response}")

        return response.strip()

    def chat(self, user_input):
        """Simple chat interface."""
        return self.generate_response(user_input)

    def reset(self):
        """Reset conversation history."""
        self.conversation_history = []
        print("Conversation reset.")

# Create chatbot instance
chatbot = NepaliGPTChatbot(model, tokenizer, device)
print("✅ Nepali GPT Chatbot created!")

In [ ]:
# Test the chatbot
print("="*60)
print("🤖 NEPALI GPT CHATBOT TEST")
print("="*60)

test_inputs = [
    "नमस्ते",
    "तपाईंको नाम के हो?",
    "नेपालको राजधानी के हो?",
    "आज मौसम कस्तो छ?"
]

for user_input in test_inputs:
    response = chatbot.chat(user_input)
    print(f"\n👤 User: {user_input}")
    print(f"🤖 Bot: {response}")

# Reset for next test
chatbot.reset()

## 4. Retrieval-Based Chatbot with Semantic Similarity

Using **NepBERTa** for computing semantic similarity between user query and predefined responses.

In [ ]:
# Load NepBERTa for embeddings
print("Loading NepBERTa for semantic similarity...")

try:
    nepberta_tokenizer = AutoTokenizer.from_pretrained("NepBERTa/NepBERTa")
    nepberta_model = AutoModel.from_pretrained("NepBERTa/NepBERTa")
    nepberta_model = nepberta_model.to(device)
    print("✅ NepBERTa loaded!")
    USE_NEPBERTA = True
except Exception as e:
    print(f"Could not load NepBERTa: {e}")
    print("Trying multilingual BERT instead...")
    nepberta_tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
    nepberta_model = AutoModel.from_pretrained("bert-base-multilingual-cased")
    nepberta_model = nepberta_model.to(device)
    print("✅ mBERT loaded as fallback!")
    USE_NEPBERTA = True

In [ ]:
# Create knowledge base for retrieval chatbot
knowledge_base = {
    # Greetings
    "नमस्ते": "नमस्ते! म तपाईंलाई कसरी मद्दत गर्न सक्छु?",
    "हेलो": "हेलो! कस्तो हुनुहुन्छ आज?",
    "के छ": "सबै ठीक छ, धन्यवाद! तपाईंको के छ?",
    "कस्तो छ": "म ठीक छु। तपाईंलाई केही सोध्नु छ?",

    # Identity
    "तपाईंको नाम के हो": "म नेपाली च्याटबोट हुँ, तपाईंको सहायक।",
    "तपाई को हो": "म एक AI च्याटबोट हुँ, नेपाली भाषामा कुराकानी गर्न बनाइएको।",
    "कसले बनायो तपाईंलाई": "म Deep Learning र Transformer प्रविधिबाट बनाइएको हुँ।",

    # Nepal Info
    "नेपालको राजधानी के हो": "नेपालको राजधानी काठमाडौं हो।",
    "नेपालको जनसंख्या कति छ": "नेपालको जनसंख्या लगभग ३ करोड छ।",
    "नेपाल कहाँ छ": "नेपाल दक्षिण एशियामा भारत र चीनको बीचमा अवस्थित छ।",
    "सगरमाथाको उचाई कति हो": "सगरमाथाको उचाई ८,८४८.८६ मिटर छ।",
    "नेपालको राष्ट्रिय फूल के हो": "नेपालको राष्ट्रिय फूल लालीगुराँस हो।",
    "नेपालको राष्ट्रिय चरा के हो": "नेपालको राष्ट्रिय चरा डाँफे हो।",

    # Food
    "दालभात के हो": "दालभात नेपालीहरूको मुख्य खाना हो - भात, दाल, तरकारी र अचार।",
    "मोमो के हो": "मोमो नेपालको लोकप्रिय खाना हो, डम्प्लिंग जस्तो।",
    "नेपाली खाना के हो": "नेपाली खानामा दालभात, मोमो, सेल रोटी, गुन्द्रुक प्रसिद्ध छन्।",

    # Festivals
    "दशैं के हो": "दशैं नेपालको सबैभन्दा ठूलो चाड हो, दुर्गा पूजाको रूपमा मनाइन्छ।",
    "तिहार के हो": "तिहार दीपावली जस्तो चाड हो, पाँच दिनसम्म मनाइन्छ।",
    "होली के हो": "होली रंगहरूको चाड हो, फागुन पूर्णिमामा मनाइन्छ।",

    # Tourism
    "पोखरा कहाँ छ": "पोखरा नेपालको पश्चिमी भागमा छ, फेवा ताल र हिमालको दृश्यको लागि प्रसिद्ध।",
    "लुम्बिनी किन प्रसिद्ध छ": "लुम्बिनी भगवान बुद्धको जन्मस्थान हो, UNESCO विश्व सम्पदा स्थल।",
    "चितवनमा के छ": "चितवनमा राष्ट्रिय निकुञ्ज छ, गैंडा र बाघ हेर्न सकिन्छ।",

    # General
    "धन्यवाद": "स्वागत छ! खुशी लाग्यो मद्दत गर्न पाएर।",
    "बाय": "अलविदा! फेरि भेटौंला!",
    "मलाई मद्दत चाहियो": "म तपाईंलाई मद्दत गर्न तयार छु। के चाहिन्छ?",

    # Weather/Time
    "आज मौसम कस्तो छ": "म मौसम जानकारी दिन सक्दिन, तर आशा गर्छु राम्रो छ!",
    "कति बज्यो": "म समय बताउन सक्दिन, तपाईंको फोन हेर्नुहोस्!",
}

print(f"Knowledge base created with {len(knowledge_base)} entries.")

In [ ]:
def get_embedding(text, tokenizer, model, device):
    """
    Get sentence embedding using BERT model.
    """
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    # Use mean pooling of last hidden state
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

def cosine_similarity(a, b):
    """Compute cosine similarity between two vectors."""
    return torch.nn.functional.cosine_similarity(a, b).item()

# Pre-compute embeddings for knowledge base
print("Computing embeddings for knowledge base...")
kb_embeddings = {}

for question in knowledge_base.keys():
    emb = get_embedding(question, nepberta_tokenizer, nepberta_model, device)
    kb_embeddings[question] = emb

print(f"✅ Computed embeddings for {len(kb_embeddings)} questions.")

In [ ]:
class NepaliRetrievalChatbot:
    """
    Retrieval-based Nepali Chatbot using semantic similarity.
    """

    def __init__(self, knowledge_base, kb_embeddings, tokenizer, model, device, threshold=0.6):
        self.knowledge_base = knowledge_base
        self.kb_embeddings = kb_embeddings
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        self.threshold = threshold
        self.default_response = "माफ गर्नुहोस्, मैले बुझिन। कृपया फेरि भन्नुहोस्।"

    def find_best_match(self, user_input):
        """
        Find the most similar question in knowledge base.
        """
        # Get embedding for user input
        user_emb = get_embedding(user_input, self.tokenizer, self.model, self.device)

        best_match = None
        best_score = -1

        for question, emb in self.kb_embeddings.items():
            score = cosine_similarity(user_emb, emb)
            if score > best_score:
                best_score = score
                best_match = question

        return best_match, best_score

    def chat(self, user_input):
        """
        Generate response based on semantic similarity.
        """
        best_match, score = self.find_best_match(user_input)

        if score >= self.threshold:
            return self.knowledge_base[best_match], best_match, score
        else:
            return self.default_response, None, score

    def get_response(self, user_input):
        """Simple response without debug info."""
        response, _, _ = self.chat(user_input)
        return response

# Create retrieval chatbot
retrieval_chatbot = NepaliRetrievalChatbot(
    knowledge_base, kb_embeddings,
    nepberta_tokenizer, nepberta_model, device,
    threshold=0.5
)

print("✅ Nepali Retrieval Chatbot created!")

In [ ]:
# Test retrieval chatbot
print("="*60)
print("🔍 RETRIEVAL-BASED CHATBOT TEST")
print("="*60)

test_queries = [
    "नमस्ते",
    "तिम्रो नाम के हो?",
    "नेपालको capital के हो?",
    "सगरमाथा कति अग्लो छ?",
    "मोमो भनेको के हो?",
    "दशैं चाड कहिले हो?",
    "पोखरामा के हेर्ने?",
    "धन्यवाद"
]

for query in test_queries:
    response, matched, score = retrieval_chatbot.chat(query)
    print(f"\n👤 User: {query}")
    print(f"🤖 Bot: {response}")
    if matched:
        print(f"   [Matched: '{matched}' | Score: {score:.3f}]")

## 5. Hybrid Chatbot (Retrieval + Generation)

Combines both approaches for better responses.

In [ ]:
class NepaliHybridChatbot:
    """
    Hybrid chatbot combining retrieval and generation.
    Uses retrieval for known queries, generation for unknown.
    """

    def __init__(self, retrieval_bot, generative_bot, retrieval_threshold=0.6):
        self.retrieval_bot = retrieval_bot
        self.generative_bot = generative_bot
        self.retrieval_threshold = retrieval_threshold
        self.conversation_history = []

    def chat(self, user_input):
        """
        Try retrieval first, fall back to generation.
        """
        # Try retrieval first
        response, matched, score = self.retrieval_bot.chat(user_input)
        method = "retrieval"

        # If retrieval confidence is low, use generation
        if score < self.retrieval_threshold:
            response = self.generative_bot.chat(user_input)
            method = "generation"

        # Store in history
        self.conversation_history.append({
            'user': user_input,
            'bot': response,
            'method': method,
            'score': score
        })

        return response, method, score

    def get_response(self, user_input):
        """Simple interface."""
        response, _, _ = self.chat(user_input)
        return response

    def reset(self):
        """Reset both chatbots."""
        self.conversation_history = []
        self.generative_bot.reset()

# Create hybrid chatbot
hybrid_chatbot = NepaliHybridChatbot(
    retrieval_chatbot,
    chatbot,  # GPT-based
    retrieval_threshold=0.55
)

print("✅ Nepali Hybrid Chatbot created!")

In [ ]:
# Test hybrid chatbot
print("="*60)
print("🔀 HYBRID CHATBOT TEST")
print("="*60)

test_inputs = [
    "नमस्ते",  # Should use retrieval
    "नेपालको राजधानी बताउनुस्",  # Should use retrieval
    "आज के गर्ने?",  # May use generation
    "तपाईं कस्तो हुनुहुन्छ?",  # May use retrieval or generation
    "हिमाल चढ्न कति गाह्रो छ?",  # Should use generation
    "धन्यवाद मद्दतको लागि",  # Should use retrieval
]

for user_input in test_inputs:
    response, method, score = hybrid_chatbot.chat(user_input)
    print(f"\n👤 User: {user_input}")
    print(f"🤖 Bot: {response}")
    print(f"   [Method: {method} | Score: {score:.3f}]")

## 6. Fine-tune GPT-2 on Custom Nepali Conversations

Let's fine-tune the pre-trained model on custom conversation data.

In [ ]:
# Create training data for fine-tuning
conversation_data = [
    "प्रयोगकर्ता: नमस्ते\nबोट: नमस्ते! म तपाईंलाई कसरी मद्दत गर्न सक्छु?",
    "प्रयोगकर्ता: तपाईंको नाम के हो?\nबोट: मेरो नाम नेपाली च्याटबोट हो।",
    "प्रयोगकर्ता: नेपालको राजधानी के हो?\nबोट: नेपालको राजधानी काठमाडौं हो।",
    "प्रयोगकर्ता: सगरमाथा कति अग्लो छ?\nबोट: सगरमाथा ८,८४८.८६ मिटर अग्लो छ।",
    "प्रयोगकर्ता: दालभात के हो?\nबोट: दालभात नेपालीहरूको मुख्य खाना हो।",
    "प्रयोगकर्ता: मोमो कस्तो हुन्छ?\nबोट: मोमो नेपालको लोकप्रिय खाना हो, डम्प्लिंग जस्तो।",
    "प्रयोगकर्ता: दशैं कहिले हुन्छ?\nबोट: दशैं असोज-कार्तिक महिनामा मनाइन्छ।",
    "प्रयोगकर्ता: तिहार के हो?\nबोट: तिहार दीपावली जस्तो पाँच दिने चाड हो।",
    "प्रयोगकर्ता: पोखरामा के छ?\nबोट: पोखरामा फेवा ताल र हिमालको दृश्य छ।",
    "प्रयोगकर्ता: लुम्बिनी किन प्रसिद्ध छ?\nबोट: लुम्बिनी बुद्धको जन्मस्थान हो।",
    "प्रयोगकर्ता: नेपालको झन्डा कस्तो छ?\nबोट: नेपालको झन्डा दुई त्रिकोणको आकारमा छ, संसारमा अद्वितीय।",
    "प्रयोगकर्ता: धन्यवाद\nबोट: स्वागत छ! खुशी लाग्यो मद्दत गर्न पाएर।",
    "प्रयोगकर्ता: बाय\nबोट: अलविदा! फेरि भेटौंला!",
    "प्रयोगकर्ता: कस्तो छ?\nबोट: म ठीक छु, धन्यवाद! तपाईं कस्तो हुनुहुन्छ?",
    "प्रयोगकर्ता: मलाई मद्दत चाहियो\nबोट: म तपाईंलाई मद्दत गर्न तयार छु। के चाहिन्छ?",
    "प्रयोगकर्ता: तपाई के गर्न सक्नुहुन्छ?\nबोट: म नेपाली भाषामा कुराकानी र जानकारी दिन सक्छु।",
    "प्रयोगकर्ता: नेपालको जनसंख्या कति छ?\nबोट: नेपालको जनसंख्या लगभग ३ करोड छ।",
    "प्रयोगकर्ता: नेपाली भाषामा कति अक्षर छन्?\nबोट: नेपाली वर्णमालामा ३६ व्यञ्जन र १२ स्वर छन्।",
]

print(f"Created {len(conversation_data)} training conversations.")

In [ ]:
# Prepare dataset for fine-tuning
def tokenize_conversations(conversations, tokenizer, max_length=256):
    """
    Tokenize conversation data for training.
    """
    tokenized_data = []

    for conv in conversations:
        # Add EOS token at the end
        text = conv + tokenizer.eos_token

        # Tokenize
        encodings = tokenizer(
            text,
            truncation=True,
            max_length=max_length,
            padding='max_length',
            return_tensors='pt'
        )

        tokenized_data.append({
            'input_ids': encodings['input_ids'].squeeze(),
            'attention_mask': encodings['attention_mask'].squeeze(),
            'labels': encodings['input_ids'].squeeze()
        })

    return tokenized_data

# Tokenize
tokenized_conversations = tokenize_conversations(conversation_data, tokenizer)

# Create HuggingFace Dataset
train_dataset = Dataset.from_list([
    {
        'input_ids': item['input_ids'].tolist(),
        'attention_mask': item['attention_mask'].tolist(),
        'labels': item['labels'].tolist()
    }
    for item in tokenized_conversations
])

print(f"Dataset created with {len(train_dataset)} samples.")

In [ ]:
# Fine-tuning configuration
training_args = TrainingArguments(
    output_dir='./nepali_chatbot_finetuned',
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=50,
    save_total_limit=2,
    logging_steps=10,
    learning_rate=5e-5,
    warmup_steps=10,
    logging_dir='./logs',
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

print("Trainer configured!")

In [ ]:
# Fine-tune the model (optional - takes time)
print("="*60)
print("FINE-TUNING NEPALI GPT-2")
print("="*60)

# Uncomment to actually fine-tune
trainer.train()
model.save_pretrained('./nepali_chatbot_finetuned')
tokenizer.save_pretrained('./nepali_chatbot_finetuned')
print("✅ Fine-tuning complete! Model saved.")

#print("\n⚠️ Fine-tuning is commented out to save time.")
#print("Uncomment the lines above to train on your data.")

## 7. Interactive Chat Interface

In [ ]:
def interactive_chat(chatbot_instance, chatbot_name="Nepali Chatbot"):
    """
    Interactive chat session.
    """
    print("\n" + "="*60)
    print(f"🤖 {chatbot_name} - Interactive Mode")
    print("="*60)
    print("Type 'quit', 'exit', or 'बाहिर' to end the conversation.")
    print("Type 'reset' to clear conversation history.")
    print("-"*60)

    while True:
        user_input = input("\n👤 You: ").strip()

        if not user_input:
            continue

        if user_input.lower() in ['quit', 'exit', 'बाहिर', 'bye', 'बाय']:
            print("🤖 Bot: अलविदा! फेरि भेटौंला!")
            break

        if user_input.lower() == 'reset':
            if hasattr(chatbot_instance, 'reset'):
                chatbot_instance.reset()
            print("🤖 Bot: Conversation reset. नयाँ कुराकानी सुरु गरौं!")
            continue

        # Get response
        if hasattr(chatbot_instance, 'get_response'):
            response = chatbot_instance.get_response(user_input)
        else:
            response = chatbot_instance.chat(user_input)
            if isinstance(response, tuple):
                response = response[0]

        print(f"🤖 Bot: {response}")

# Uncomment to start interactive chat

interactive_chat(hybrid_chatbot, "Nepali Hybrid Chatbot")

In [ ]:
# Demo conversation
print("\n" + "="*60)
print("🇳🇵 DEMO CONVERSATION")
print("="*60)

demo_conversation = [
    "नमस्ते",
    "तपाईंको नाम के हो?",
    "नेपालको बारेमा केही बताउनुस्",
    "सगरमाथा कति अग्लो छ?",
    "नेपाली खाना के राम्रो छ?",
    "धन्यवाद, बाय!"
]

hybrid_chatbot.reset()

for user_input in demo_conversation:
    response = hybrid_chatbot.get_response(user_input)
    print(f"\n👤 User: {user_input}")
    print(f"🤖 Bot: {response}")

## 8. Save the Models

In [ ]:
import pickle
import json

# Save knowledge base
with open('nepali_knowledge_base.json', 'w', encoding='utf-8') as f:
    json.dump(knowledge_base, f, ensure_ascii=False, indent=2)

print("✅ Knowledge base saved to nepali_knowledge_base.json")

# Save conversation data
with open('nepali_conversations.json', 'w', encoding='utf-8') as f:
    json.dump(conversation_data, f, ensure_ascii=False, indent=2)

print("✅ Conversation data saved to nepali_conversations.json")

## 9. Summary

### Pre-trained Models Used:

| Model | Purpose | Source |
|-------|---------|--------|
| **Sakonii/distilgpt2-nepali** | Text generation | HuggingFace |
| **NepBERTa/NepBERTa** | Semantic embeddings | HuggingFace |
| **bert-base-multilingual-cased** | Fallback embeddings | HuggingFace |

### Chatbot Types Created:

| Type | Approach | Best For |
|------|----------|----------|
| **Generative** | GPT-2 text generation | Open-ended conversations |
| **Retrieval** | Semantic similarity | Known Q&A pairs |
| **Hybrid** | Both combined | General use |

### Key Features:
- Uses **actual pre-trained Nepali models**
- Semantic similarity with NepBERTa/mBERT
- Conversation history tracking
- Fine-tuning capability
- Knowledge base extensibility

In [ ]:
print("\n" + "="*60)
print("✅ NEPALI CHATBOT NOTEBOOK COMPLETE!")
print("="*60)
print("\nYou have learned:")
print("  ✓ Loading pre-trained Nepali GPT-2 model")
print("  ✓ Building generative chatbot")
print("  ✓ Building retrieval-based chatbot with semantic similarity")
print("  ✓ Creating hybrid chatbot")
print("  ✓ Fine-tuning on custom data")
print("\n🇳🇵 धन्यवाद! (Thank you!)")